# Welcome!

This example shows a very basic usage case of MultiBench. In particular, it demonstrates how to use MultiBench with the affective computing dataset MOSI, and how to use it with a very simple fusion model. Alos, we show how to use Comic Mischief and Hate Speech datasets with this toolkit

While this will be simple, it will show off most of the capabilities of MultiBench, and most of the conventions at the heart of the system.

To begin, let's clone the repo and setup our interpreter to run commands inside the folder.

## Comic Mischief and Hate Speech Datasets



In [1]:
!git clone https://github.com/iltocl/dcc-tutorial-multizoo-multibench.git

Cloning into 'dcc-tutorial-multizoo-multibench'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 125 (delta 62), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (125/125), 1.16 MiB | 7.33 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
!git clone https://github.com/pliang279/MultiBench.git

Cloning into 'MultiBench'...
remote: Enumerating objects: 6943, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6943 (delta 72), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6943/6943), 51.07 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.


In [3]:
!pip install -U memory_profiler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Try to download the data folder with MOSI, MOSEI, SARCASM and MUSTARD files using the below command. If this does not work for you, please download the data file locally, and upload it to the folder "/content/MultiBench/data/"

In [5]:
%cd dcc-tutorial-multizoo-multibench
import torch
import sys
import os
from data_loader import get_loader

/content/dcc-tutorial-multizoo-multibench


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

In [6]:
DATA_PATH = '/content/drive/MyDrive/ComicHate/hsdv_batch_250'
dataset = 'Hate' #'CoMic'
batch_size = 16
max_pad = 70

train_data, len1 = get_loader(data = dataset, data_dir = DATA_PATH, mode = 'train', task = 'Binary', batch_size = batch_size,
                              max_pad = max_pad, words = True, shuffle = True)

val_data, len2 = get_loader(dataset, data_dir = DATA_PATH, mode = 'val', task = 'Binary', batch_size = batch_size,
                            max_pad = max_pad, words = True, shuffle = True)

test_data, len3 = get_loader(dataset, data_dir = DATA_PATH, mode = 'test', task = 'Binary', batch_size = batch_size,
                             max_pad = max_pad, words = True, shuffle = False)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


val


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


test


In [7]:
%cd ..
%cd MultiBench
from unimodals.common_models import GRU, MLP, Sequential, Identity, VGG, Linear, LSTM

# As this example is meant to be simple and easy to train, we'll pass in identity
# functions for each of the modalities in MOSI:
encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]
#encoders = [Linear(128, 128).cuda(), VGG(128).cuda(), LSTM(128, 128).cuda()]

/content
/content/MultiBench


In [8]:
# Import a fusion paradigm, in this case early concatenation.
from fusions.common_fusions import ConcatEarly  # noqa

# Initialize the fusion module
fusion = ConcatEarly().cuda()

In [10]:
input_dim = 1920 #128 * 3 #1920
head = Sequential(GRU(input_dim, 512, dropout=True, has_padding=False,
                  batch_first=True, last_only=True), MLP(512, 512, 1)).cuda()

In [15]:
# Standard supervised learning training loop
from training_structures.Supervised_Learning import train, test

# For more information regarding parameters for any system, feel free to check out the documentation
# at multibench.readthedocs.io!
train(encoders, fusion, head, train_data, val_data, 5, task="classification", optimtype=torch.optim.AdamW,
      is_packed=False, lr=1e-3, save='hate_ef_r0.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

print("Testing:")
model = torch.load('hate_ef_r0.pt').cuda()
test(model, test_data, 'affect', is_packed=False,
     criterion=torch.nn.L1Loss(), task="classification", no_robust=True)

Epoch 0 train loss: tensor(0.3006, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: tensor(0.2631, device='cuda:0') acc: 0.7391304347826086
Saving Best
Epoch 1 train loss: tensor(0.2947, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: tensor(0.2618, device='cuda:0') acc: 0.7391304347826086
Epoch 2 train loss: tensor(0.2898, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: tensor(0.2666, device='cuda:0') acc: 0.7391304347826086
Epoch 3 train loss: tensor(0.2885, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: tensor(0.2647, device='cuda:0') acc: 0.7391304347826086
Epoch 4 train loss: tensor(0.2804, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: tensor(0.2665, device='cuda:0') acc: 0.7391304347826086
Training Time: 120.92961263656616
Training Peak Mem: 5209.9609375
Training Params: 4001793
Testing:
acc: 0.72
Inference Time: 0.7643392086029053
Inference Params: 4001793


## Affect Datasets

In [16]:
!pip install gdown && gdown https://drive.google.com/drive/folders/1drao0aXgS1tPCr55riqMLfo0pMPzDAR-?usp=sharing -O /content/MultiBench/ --folder

Retrieving folder contents
Processing file 1i8ePqY6HQ3m4joa5bjNRj5q98F9Bo4lF humor.pkl
Processing file 1uhUloaigNUMYU7Wq_EZj58pCGIoto34T mosei_raw.pkl
Processing file 13McsNwX1_kTnA3i2KyDWlLWnh52Kvz4V mosi_raw.pkl
Processing file 1dN7AT3ytEnCgL18FnKN-jXN4GkEQFKZS sarcasm.pkl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1i8ePqY6HQ3m4joa5bjNRj5q98F9Bo4lF
From (redirected): https://drive.google.com/uc?id=1i8ePqY6HQ3m4joa5bjNRj5q98F9Bo4lF&confirm=t&uuid=a2f2933c-3b6f-47f6-ad3d-25862b467048
To: /content/MultiBench/data/humor.pkl
100% 1.22G/1.22G [00:27<00:00, 45.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1uhUloaigNUMYU7Wq_EZj58pCGIoto34T
From (redirected): https://drive.google.com/uc?id=1uhUloaigNUMYU7Wq_EZj58pCGIoto34T&confirm=t&uuid=9fb27b9f-d7fa-48a4-852c-270d453bb217
To: /content/MultiBench/data/mosei_raw.pkl
100% 9.94G/9.94G [03:01<00:00,

As Colab famously has bad handling of Conda env files, we'll install the dependencies manually so that it works. Please note that other systems might require installation of a long list of other dependencies.

From here, let's import some of MultiBench and get working:

In [17]:
import torch
import sys
import os

First, we'll import and create the dataloader for the MOSI dataset, which we're working with:

In [55]:
# Import the associated dataloader for affect datasets, which MOSI is a part of.
from datasets.affect.get_data import get_dataloader

# Create the training, validation, and test-set dataloaders.
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/data/humor.pkl', robust_test=False, max_pad=True, data_type='humor', max_seq_len=50)

Then, let's define our MultiModal model to test. MultiBench divides models into three separate portions.

Firstly, let's define the encoders of the raw modality information, which come from the "unimodals" section of MultiBench:

In [56]:
# Here, we'll import several common modules should you want to mess with this more.
from unimodals.common_models import GRU, MLP, Sequential, Identity, VGG, Linear, LSTM

# As this example is meant to be simple and easy to train, we'll pass in identity
# functions for each of the modalities in MOSI:
encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]
#encoders = [Linear(371, 128).cuda(), Linear(81, 128).cuda(), Linear(300, 128).cuda()]

Then, let's define the fusion paradigm, which will govern how we take the current modalities, and combine them.

For this example, we'll use the ConcatEarly fusion, which just concatenates the inputs along the second dimension.

In [57]:
# Import a fusion paradigm, in this case early concatenation.
from fusions.common_fusions import ConcatEarly  # noqa

# Initialize the fusion module
fusion = ConcatEarly().cuda()

Lastly, we'll define a 'head' module, which takes the output of the fusion module, and applies transformations to get an output that correponds to our problem - sarcasm detection.

In [60]:
head = Sequential(GRU(752, 512, dropout=True, has_padding=False,
                  batch_first=True, last_only=True), MLP(512, 512, 1)).cuda()

And with that, we're almost done! Now we just need to put them into one of MultiBench's training loops, and set it running:

In [ ]:
# Standard supervised learning training loop
from training_structures.Supervised_Learning import train, test

# For more information regarding parameters for any system, feel free to check out the documentation
# at multibench.readthedocs.io!
train(encoders, fusion, head, traindata, validdata, 5, task="classification", optimtype=torch.optim.AdamW,
      is_packed=False, lr=1e-3, save='Multibench/humor_ef_r0.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

print("Testing:")
model = torch.load('humor_ef_r0.pt').cuda()
test(model, testdata, 'affect', is_packed=False,
     criterion=torch.nn.L1Loss(), task="classification", no_robust=True)

And with that, you've taken your first step into using MultiBench! We hope you find the library useful, and feel free to make an issue on GitHub should there be any confusions regarding how to use an aspect of the package.